In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
yrs=[1983,1984,1985]
arr=[]
links=[]
for yr in yrs:
    html=requests.get("https://www.profootballarchives.com/"+str(yr)+"usfl.html")
    soup=BeautifulSoup(html.text)
    stats=soup.find("div",{"id":"stats"})
    table=stats.find_all("table")[0]
    for a in table.find_all("a"):
        links.append(a.get("href"))
        url="https://www.profootballarchives.com/"+str(a.get("href"))
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[4]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(yr)
            arr.append(h)
yrs=[2022]
links=[]
html=requests.get("https://www.profootballarchives.com/2022usflbir.html")
soup=BeautifulSoup(html.text)
table=soup.find_all("table")[3]
for a in table.find_all("a"):
    if '<a></a>' not in str(a):
        links.append(a.get("href"))
for link in links:
    for yr in yrs:
        
        link=str(link)
        if '83' in link:
            link=link.split("83")
            link=link[1]
        if '84' in link:
            link=link.split("84")
            link=link[1]
        if '85' in link:
            link=link.split("85")
            link=link[1]
        link=link
        url="https://www.profootballarchives.com/"+link
        html=requests.get(url)
        soup=BeautifulSoup(html.text)
        
        link=soup.find("title")
        link=str(link.text)
        team=link.split(str(yr))
        team=team[1]
        team=team.replace(" (USFL)","")
        team=team.replace(" - Pro Football Archives","")
        table=soup.find_all("table")[3]
        
        for tr in table.find_all("tr"):
            h=[]
            for td in tr.find_all("td"):
                h.append(td.text)
                
            h.append(team)
            h.append(2022)
            arr.append(h)
df=pd.concat([pd.DataFrame(arr)])
df.columns=['Day','Date','VenueType','team2','score','result','OT','Location','Venue','Attendance','team1','season']
df=df.astype({'VenueType':'string'})

df=df[df.Date.notna()]
df=df[df.score.notna()]
df['score2']=0
df['score1']=0
for row in df.itertuples():
    score=str(row.score)
    score=score.split("-")
    s1=score[0]
    s2=score[1]
    df.at[row.Index,'score1']=s1
    df.at[row.Index,'score2']=s2
df['Date']=df['Date'].str.replace("/","")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,"Date":"string"})  
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)


In [1]:
f

NameError: name 'f' is not defined

In [76]:
import math
import numpy as np
import pandas as pd
df=pd.read_csv("https://github.com/easella/usfl/raw/main/usflscores.csv")
df=df.astype({"team1":"string","team2":"string",'score2':float,'score1':float,'Date':"string"})
df['team1']=df['team1'].str.replace(" ","")
df['team2']=df['team2'].str.replace(" ","")
for row in df.itertuples():
    f=df[(df.Date==row.Date)&(df.team2==row.team1)&(df.team1==row.team2)]
    if len(f)>0:
        df=df.drop(row.Index)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
hfa=[]
n=df[df.VenueType!='N']
for row in n.itertuples():
    if row.VenueType=='A':
        hfa.append(row.score2-row.score1)
    else:
        hfa.append(row.score1-row.score2)
homef=round(np.mean(hfa))
w=0

df.sort_values(by='Date',inplace=True)
l=0
class Elo:
  global winnerHome
  def __init__(self,k,g=1,homefield = 182.5):
    self.ratingDict   = {}  
    self.k        = k
    self.g        = g
  def addPlayer(self,name,rating = 1500):
    self.ratingDict[name] = rating
  def gameOver(self, winner, loser, winnerHome,neutral,wins,loses):
    homef=50
    wb=0
    lb=0
    if winnerHome and neutral!='N':
      result = self.expectResult(self.ratingDict[winner] + homef, self.ratingDict[loser])
    if winnerHome==False and neutral!='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser] + homef)
    if neutral=='N':
      result = self.expectResult(self.ratingDict[winner], self.ratingDict[loser])
    wins=float(wins)
    loses=float(loses)
    if wins==loses:
        tie=True
    else:
        tie=False
    if tie:
        mult=(math.log(0+1))*(2.2/1*0.001+2.2)
        shift=(self.k*mult)*(0.5 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
    if tie !=True:
        win=eloLeague.ratingDict[winner]
        lose=eloLeague.ratingDict[loser]
        if neutral==True:
          win=(eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome and neutral!=True:
          win=(homef+eloLeague.ratingDict[winner])
          lose=(eloLeague.ratingDict[loser])
        if winnerHome!=True and neutral!=True:
          lose=(homef+eloLeague.ratingDict[loser])
          win=(eloLeague.ratingDict[winner])
        global zy
        if(win>lose):
          zy=(win-lose)
        if(win==lose):
          zy=(win-lose)
        if(win<lose):
          zy=(win-lose)
        mult=(math.log((wins-loses)+1))*(2.2/(zy)*0.001+2.2)
        shift=(self.k*mult)*(1 - result)  
        self.ratingDict[winner] +=shift
        self.ratingDict[loser]  -=shift
  def expectResult(self, p1, p2):
        global exp
        exp = (p2-p1)/400
        n2=10
        j=exp
        o=1/((n2**(j))+1)
        return float(o)
teams=set(df.team1.tolist()+df.team2.tolist())

eloLeague=Elo(k=20)
currSeason=1983
q=pd.read_html("https://en.wikipedia.org/wiki/2023_USFL_season")[1]
q.columns = q.columns.droplevel(1)
q=q.replace("[a]","")
q=q.rename(columns={'Team[10]':'Team'})
q=q.rename(columns={'Home stadium':'Venue'})
venues={}
venues[2023]={}
for row in q.itertuples():
    team=str(row.Team)
    team=team.replace("[a]","")
    team=team.replace(" ","")
    team=team.replace("MemphisShowboats","TampaBayBandits")
    venues[2023][team]=row.Venue


    
q=pd.read_html("https://en.wikipedia.org/wiki/United_States_Football_League_(2022)#Teams")[2]
q.columns = q.columns.droplevel(1)
q=q.replace("[a]","")
q=q.rename(columns={'Team[72]':'Team'})
q=q.rename(columns={'Stadium':'Venue'})
venues[2022]={}
for row in q.itertuples():
    team=str(row.Team)
    team=team.replace("[a]","")
    team=team.replace(" ","")
    team=team.replace("MemphisShowboats","TampaBayBandits")

    venues[2022][team]=row.Venue
t=pd.read_csv("https://ontheroadtovote.com/ncaab/usflscores.txt")
df=pd.concat([df,t])
for row in df.itertuples():
    if row.season>2021:
        if venues[row.season][row.team2]==venues[row.season][row.team1]:
            df.at[row.Index,'VenueType']='N'
df=df.drop(['Unnamed: 0'], axis=1)
df=df.drop_duplicates()
for team in teams:
    eloLeague.addPlayer(team,rating=1500)
for row in df.itertuples():
    if float(row.season)>currSeason:
        season=float(row.season)
        lsdf=df[df.season<season]
        ls=set(lsdf.team1.tolist()+lsdf.team2.tolist())
        csdf=df[df.season==season]
        cs=set(csdf.team1.tolist()+csdf.team2.tolist())
        
                
       
            
       
        
        
        for key in eloLeague.ratingDict.keys():
            
            if(key  in ls and key in cs):
                
                pre=eloLeague.ratingDict[key]
                eloLeague.ratingDict[key]=(eloLeague.ratingDict[key]*(2/3))+(1505*(1/3))
                af=eloLeague.ratingDict[key]
    
                
        currSeason+=1
    if row.score1>row.score2 or row.score1==row.score2:
        if row.VenueType=='A':
            hfa=0
            if row.VenueType!='N':
                hfa=65
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=False
        else:
            hfa=0
            if row.VenueType!='N':
                hfa=65
            if (eloLeague.ratingDict[row.team1]+hfa)>(eloLeague.ratingDict[row.team2]):
                w+=1
            if (eloLeague.ratingDict[row.team1]+hfa)<(eloLeague.ratingDict[row.team2]):
                l+=1
            winnerhome=True
        eloLeague.gameOver(row.team1,row.team2,winnerhome,row.VenueType,row.score1,row.score2)
    if row.score1<row.score2:
        if row.VenueType=='A':
            hfa=0
            if row.VenueType!='N':
                hfa=65
            if (eloLeague.ratingDict[row.team1])<(eloLeague.ratingDict[row.team2]+hfa):
                w+=1
            if (eloLeague.ratingDict[row.team1])>(eloLeague.ratingDict[row.team2]+hfa):
                l+=1
            winnerhome=True
        else:
            winnerhome=False
        eloLeague.gameOver(row.team2,row.team1,winnerhome,row.VenueType,row.score2,row.score1)
css=df[df.season==2023]
if len(css)==0:
    lsdf=df[df.season<2023]
    ls=set(lsdf.team1.tolist()+lsdf.team2.tolist())
    csdf=df[df.season<2023]
    cs=set(csdf.team1.tolist()+csdf.team2.tolist()) 
    for key in eloLeague.ratingDict.keys():
            
        if(key  in ls and key in cs):
            pre=eloLeague.ratingDict[key]
            eloLeague.ratingDict[key]=(eloLeague.ratingDict[key]*(2/3))+(1500*(1/3))
            af=eloLeague.ratingDict[key]
            print(pre,af,key)
    

In [75]:
df

,Day,Date,VenueType,team2,score,result,OT,Location,Venue,Attendance,team1,season,score2,score1
107,Sun,1983-03-06 00:00:00,A,WashingtonFederals,28-7,W,NaN,"Washington, DC",RFK Stadium,38010.0,ChicagoBlitz,1983.0,7.0,28.0
105,Sun,1983-03-06 00:00:00,H,NewJerseyGenerals,20-15,W,NaN,"Los Angeles, CA",Memorial Coliseum,34002.0,LosAngelesExpress,1983.0,15.0,20.0
108,Sun,1983-03-06 00:00:00,H,PhiladelphiaStars,7-13,L,NaN,"Denver, CO",Mile High Stadium,45102.0,DenverGold,1983.0,13.0,7.0
109,Sun,1983-03-06 00:00:00,H,BostonBreakers,21-17,W,NaN,"Tampa, FL",Tampa Stadium,42437.0,TampaBayBandits,1983.0,17.0,21.0
106,Sun,1983-03-06 00:00:00,H,OaklandInvaders,0-24,L,NaN,"Tempe, AZ",Sun Devil Stadium,45167.0,ArizonaWranglers,1983.0,24.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,Sat,2022-06-25 00:00:00,N,BirminghamStallions,17-31,L,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,NewOrleansBreakers,2022.0,31.0,17.0
472,Sun,2022-07-03 00:00:00,H,BirminghamStallions,33-30,W,NaN,"Canton, OH",Tom Benson Hall of Fame Stadium,NaN,PhiladelphiaStars,2022.0,30.0,33.0
0,NaN,2023-04-15,A,PhiladelphiaStars,27-23,NaN,NaN,NaN,NaN,NaN,TampaBayBandits,2023.0,NaN,NaN
1,NaN,2023-04-15,A,NewJerseyGenerals,10-27,NaN,NaN,NaN,NaN,NaN,BirminghamStallions,2023.0,NaN,NaN


In [66]:
venues[2023]['N']

{'MichiganPanthers': 'Ford Field',
 'PhiladelphiaStars': 'Ford Field',
 'NewJerseyGenerals': 'Tom Benson Hall of Fame Stadium',
 'PittsburghMaulers': 'Tom Benson Hall of Fame Stadium',
 'SouthDivision': 'South Division',
 'BirminghamStallions': 'Protective Stadium',
 'NewOrleansBreakers': 'Protective Stadium',
 'HoustonGamblers': 'Simmons Bank Liberty Stadium',
 'TampaBayBandits': 'Simmons Bank Liberty Stadium'}

In [15]:
l2s={k: v for k, v in sorted(eloLeague.ratingDict.items(), key=lambda item: item[1],reverse=True)}
for key in l2s.keys():
    print(key,eloLeague.ratingDict[key])

BaltimoreStars 1705.1038997166688
OaklandInvaders 1679.696295527269
PhiladelphiaStars 1643.6272877288447
ArizonaWranglers 1641.478882490435
MemphisShowboats 1634.9906524614637
BostonBreakers 1579.1715291463317
BirminghamStallions 1579.1227404234091
JacksonvilleBulls 1568.4663340766829
NewJerseyGenerals 1547.6171168472488
HoustonGamblers 1528.035564754844
ArizonaOutlaws 1495.1158625793241
MichiganPanthers 1474.6240350569353
DenverGold 1472.5784696901728
NewOrleansBreakers 1441.9185108818672
TampaBayBandits 1424.83749143576
OrlandoRenegades 1419.7242188518012
ChicagoBlitz 1404.812582481728
PortlandBreakers 1400.1523634157047
PittsburghMaulers 1386.8839564006846
SanAntonioGunslingers 1378.2340442807636
WashingtonFederals 1323.3467752685046
OklahomaOutlaws 1296.4900538894267
LosAngelesExpress 1296.087420804568
